In [24]:
import pandas as pd
import numpy as np
import boto3
import psycopg2
import configparser

In [25]:
config = configparser.ConfigParser()
config.read('escec.cfg')

['escec.cfg']

In [26]:
RDS_HOST = 'comercio-db.cxfxom467xtx.us-east-1.rds.amazonaws.com'
#RDS_HOST=config.get('DW', 'RDS_HOST')
#RDS_HOST
print(RDS_HOST)

comercio-db.cxfxom467xtx.us-east-1.rds.amazonaws.com


In [27]:
postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""  

#### Creación de dimensiones 

In [28]:
sql_query = 'SELECT * FROM customer;'
dimcustomer = pd.read_sql(sql_query, postgres_driver)
dimcustomer.head()

,customersk,customer_id,name,email,phone,customer_city,customer_state
0,2610,06b8999e2fba1a1fbc88172c00ba8bc7,Raquel Judd,Raquel_Judd2052@iscmr.app,Raquel Judd,franca,SP
1,9561,18955e83d337fd6b2def6b18a428ac77,Sharon Walsh,Sharon_Walsh658@yfxpw.shop,Sharon Walsh,sao bernardo do campo,SP
2,30460,4e7b3e00288586ebd08712fdd0374a03,Marvin Radcliffe,Marvin_Radcliffe1768@bu2lo.directory,Marvin Radcliffe,sao paulo,SP
3,69605,b2b6027bc5c5109e529d4dc6358b12c3,Javier Stone,Javier_Stone 9704@karnv.digital,Javier Stone,mogi das cruzes,SP
4,30707,4f2d8ab171c80ec8364f7c12e35b23ad,Carl Palmer,Carl_Palmer3085@qu9ml.info,Carl Palmer,campinas,SP


In [29]:
sql_query = 'SELECT * FROM seller;'
dimseller = pd.read_sql(sql_query, postgres_driver)
dimseller.head()

,sellersk,seller_id,name,email,phone,seller_city,seller_state
0,622,3442f8959a84dea7ee197c632cb2df15,Harry Bristow,Harry_Bristow7332@qu9ml.mobi,3-344-234-7821,campinas,SP
1,2540,d1b65fc7debc3361ea86b5f14c68d2e2,Rylee Leslie,Rylee_Leslie802@hepmv.com,0-613-821-5215,mogi guacu,SP
2,2505,ce3ad9de960102d0677a81f5d0bb7b2d,Melania Price,Melania_Price7438@mpibr.info,1-012-871-2736,rio de janeiro,RJ
3,2325,c0f3eea2e14555b6faeea3dd58c1b1c3,Bryon Holt,Bryon_Holt9825@cke3u.software,5-082-413-8028,sao paulo,SP
4,981,51a04a8a6bdcb23deccc82b0b80742cf,Carolyn Furnell,Carolyn_Furnell3693@1wa8o.business,7-014-178-0073,braganca paulista,SP


In [30]:
sql_query = 'SELECT * FROM review;'
dimreview = pd.read_sql(sql_query, postgres_driver)
dimreview.head()

,reviewsk,review_id,review_score,review_comment_message
0,47621,7bc2406110b926393aa56f80a40eba40,4,sem comentarios
1,49619,80e641a11e56f04c1ad469d5645fdfde,5,sem comentarios
2,13223,228ce5500dc1d8e020d8d1322874b6f0,5,sem comentarios
3,88592,e64fb393e7b32834bb789ff8bb30750e,5,Recebi bem antes do prazo estipulado.
4,95219,f7c4243c7fe1938f181bec41a392bdeb,5,Parabéns lojas lannister adorei comprar pela I...


In [31]:
sql_query = 'SELECT * FROM orderhead;'
dimorder = pd.read_sql(sql_query, postgres_driver)
dimorder.head()

,ordersk,order_id,order_status
0,88950,e481f51cbdc54678b7cc49136f2d6af7,delivered
1,32545,53cdb2fc8bc7dce0b6741e2150273451,delivered
2,27769,47770eb9100c2d0c44946d9cf07ec65d,delivered
3,57385,949d5b44dbf5de918fe9c16f97b45f8a,delivered
4,67043,ad21c59c0840e6cb83a9ceb5573f8159,delivered


In [32]:
sql_query = 'SELECT * FROM product;'
dimproduct= pd.read_sql(sql_query, postgres_driver)
dimproduct.head()

,productsk,product_id,product_width_cm,product_length_cm,product_height_cm,product_weight_g
0,4053,1e9e8ef04dbcff4541ed26657ea517e5,14.0,16.0,10.0,225.0
1,7645,3aa071139cb16b67ca9e5dea641aaa2f,20.0,30.0,18.0,1000.0
2,19351,96bd76ec8810374ed1b65e291975717f,15.0,18.0,9.0,154.0
3,26672,cef67bcfe19066a932b7673e239eb23d,26.0,26.0,4.0,371.0
4,20305,9dc1a7de274444849c219cff195d0b71,13.0,20.0,17.0,625.0


In [33]:
sql_query = 'SELECT order_purchase_timestamp FROM ventas;'
dimFechaHora = pd.read_sql(sql_query, postgres_driver)
dimFechaHora.head()

,order_purchase_timestamp
0,2017-10-02 10:56:33
1,2017-10-02 10:56:33
2,2017-10-02 10:56:33
3,2018-07-24 20:41:37
4,2018-08-08 08:38:49


In [34]:
dimFechaHora['order_purchase_timestamp']=pd.to_datetime(dimFechaHora['order_purchase_timestamp'])
dimFechaHora['year'] = pd.DatetimeIndex(dimFechaHora['order_purchase_timestamp']).year
dimFechaHora['month'] = pd.DatetimeIndex(dimFechaHora['order_purchase_timestamp']).month
dimFechaHora['quarter'] = pd.DatetimeIndex(dimFechaHora['order_purchase_timestamp']).quarter
dimFechaHora['day'] = pd.DatetimeIndex(dimFechaHora['order_purchase_timestamp']).day
dimFechaHora['week'] = pd.DatetimeIndex(dimFechaHora['order_purchase_timestamp']).week
dimFechaHora['dayofweek'] = pd.DatetimeIndex(dimFechaHora['order_purchase_timestamp']).dayofweek
dimFechaHora['hour'] = pd.DatetimeIndex(dimFechaHora['order_purchase_timestamp']).hour
dimFechaHora['minute'] = pd.DatetimeIndex(dimFechaHora['order_purchase_timestamp']).minute
dimFechaHora['is_weekend'] = dimFechaHora['dayofweek'].apply(lambda x: 1 if x > 5 else 0)
dimFechaHora.head()

C:\Users\50259\AppData\Local\Temp\ipykernel_9088\1286954157.py:6: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  dimFechaHora['week'] = pd.DatetimeIndex(dimFechaHora['order_purchase_timestamp']).week


,order_purchase_timestamp,year,month,quarter,day,week,dayofweek,hour,minute,is_weekend
0,2017-10-02 10:56:33,2017,10,4,2,40,0,10,56,0
1,2017-10-02 10:56:33,2017,10,4,2,40,0,10,56,0
2,2017-10-02 10:56:33,2017,10,4,2,40,0,10,56,0
3,2018-07-24 20:41:37,2018,7,3,24,30,1,20,41,0
4,2018-08-08 08:38:49,2018,8,3,8,32,2,8,38,0


In [35]:
dimFechaHora['dimfecha'] = dimFechaHora['year'].astype(str) + dimFechaHora['month'].astype(str)
dimFechaHora['dimfecha'] = dimFechaHora['dimfecha'].astype(str) + dimFechaHora['day'].astype(str)
dimFechaHora['dimfecha'] = dimFechaHora['dimfecha'].astype(str) + dimFechaHora['hour'].astype(str)
dimFechaHora['dimfecha'] = dimFechaHora['dimfecha'].astype(str) + dimFechaHora['minute'].astype(str)
dimFechaHora.drop_duplicates(inplace=True)
dimFechaHora.head()

,order_purchase_timestamp,year,month,quarter,day,week,dayofweek,hour,minute,is_weekend,dimfecha
0,2017-10-02 10:56:33,2017,10,4,2,40,0,10,56,0,20171021056
3,2018-07-24 20:41:37,2018,7,3,24,30,1,20,41,0,20187242041
4,2018-08-08 08:38:49,2018,8,3,8,32,2,8,38,0,201888838
5,2017-11-18 19:28:06,2017,11,4,18,46,5,19,28,0,201711181928
6,2018-02-13 21:18:39,2018,2,1,13,7,1,21,18,0,20182132118


In [36]:
sql_query = '''SELECT * FROM ventas;'''
df_factTable = pd.read_sql(sql_query, postgres_driver)
df_factTable.head()

,id_detalle,order_purchase_timestamp,price,payment_value,order_item_id,ordersk,customersk,paymentsk,reviewsk,productsk,sellersk,productcatesk
0,1,2017-10-02 10:56:33,29.99,18.12,1,88950,61760,1,63691.0,17417,630,72
1,2,2017-10-02 10:56:33,29.99,2.00,1,88950,61760,4,63691.0,17417,630,72
2,3,2017-10-02 10:56:33,29.99,18.59,1,88950,61760,4,63691.0,17417,630,72
3,4,2018-07-24 20:41:37,118.70,141.46,1,32545,68729,0,54401.0,11598,477,62
4,5,2018-08-08 08:38:49,159.90,179.12,1,27769,25513,1,88951.0,21798,857,8


In [37]:
df_factTable['order_purchase_timestamp']=pd.to_datetime(df_factTable['order_purchase_timestamp'])
df_factTable['year'] = pd.DatetimeIndex(df_factTable['order_purchase_timestamp']).year
df_factTable['month'] = pd.DatetimeIndex(df_factTable['order_purchase_timestamp']).month
df_factTable['day'] = pd.DatetimeIndex(df_factTable['order_purchase_timestamp']).day
df_factTable['hour'] = pd.DatetimeIndex(df_factTable['order_purchase_timestamp']).hour
df_factTable['minute'] = pd.DatetimeIndex(df_factTable['order_purchase_timestamp']).minute
df_factTable['dimfechafk'] = df_factTable['year'].astype(str) + df_factTable['month'].astype(str)
df_factTable['dimfechafk'] = df_factTable['dimfechafk'].astype(str) + df_factTable['day'].astype(str)
df_factTable['dimfechafk'] = df_factTable['dimfechafk'].astype(str) + df_factTable['hour'].astype(str)
df_factTable['dimfechafk'] = df_factTable['dimfechafk'].astype(str) + df_factTable['minute'].astype(str)
factTable_transacciones = df_factTable.drop(['year', 'month', 'day', 'hour', 'minute', 'order_purchase_timestamp'], axis=1)
factTable_transacciones.head()

,id_detalle,price,payment_value,order_item_id,ordersk,customersk,paymentsk,reviewsk,productsk,sellersk,productcatesk,dimfechafk
0,1,29.99,18.12,1,88950,61760,1,63691.0,17417,630,72,20171021056
1,2,29.99,2.00,1,88950,61760,4,63691.0,17417,630,72,20171021056
2,3,29.99,18.59,1,88950,61760,4,63691.0,17417,630,72,20171021056
3,4,118.70,141.46,1,32545,68729,0,54401.0,11598,477,62,20187242041
4,5,159.90,179.12,1,27769,25513,1,88951.0,21798,857,8,201888838


##### Lectura de Archivo desde S3

In [38]:
s3 = boto3.resource(
    service_name = 's3',
    region_name = 'us-east-1',
    aws_access_key_id = config.get('IAM', 'ACCESS_KEY'),
    aws_secret_access_key = config.get('IAM', 'SECRET_ACCESS_KEY')
)

In [39]:
for bucket in s3.buckets.all():
    S3_BUCKET_NAME = bucket.name
    print(bucket.name)

cdp-18001404


In [40]:
S3_BUCKET_NAME = 'cdp-18001404'

In [41]:
#extraemos todo lo que está en el bucket
remoteFileList = []
for objt in s3.Bucket(S3_BUCKET_NAME).objects.all():
    remoteFileList.append(objt.key)

remoteFileList

['paymentype.xlsx', 'productcate.xlsx']

##### leemos data de S3

In [42]:
import io

try:
    file = s3.Bucket(S3_BUCKET_NAME).Object('paymentype.xlsx').get()
    data = file['Body'].read()
    dimpaymentype = pd.read_excel(io.BytesIO(data), engine='openpyxl')
    file = s3.Bucket(S3_BUCKET_NAME).Object('productcate.xlsx').get()
    data = file['Body'].read()
    dimproductcate = pd.read_excel(io.BytesIO(data), engine='openpyxl')
except Exception as ex:
    print("No es un archivo.")
    print(ex)

##### Creación de Instancias en AWS para DW

In [43]:
config = configparser.ConfigParser()
config.read('escec.cfg')

['escec.cfg']

In [44]:
aws_conn = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                    aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                    region_name='us-east-1')
print(aws_conn)

In [45]:
rdsInstanceIds = []

response = aws_conn.describe_db_instances()
for resp in response['DBInstances']:
    rdsInstanceIds.append(resp['DBInstanceIdentifier'])
    db_instance_status = resp['DBInstanceStatus']

print(f"DBInstanceIds {rdsInstanceIds}")

DBInstanceIds ['comercio-db']


In [46]:
rdsIdentifier = 'dw-db'

In [47]:
try:
    response = aws_conn.create_db_instance(
            AllocatedStorage=10,
            DBName=config.get('RDS_MYSQL', 'DB_NAME'),
            DBInstanceIdentifier=rdsIdentifier,
            DBInstanceClass="db.t3.micro",
            Engine="mysql",
            MasterUsername=config.get('RDS_MYSQL', 'DB_USER'),
            MasterUserPassword=config.get('RDS_MYSQL', 'DB_PASSWORD'),
            Port=int(config.get('RDS_MYSQL', 'DB_PORT')),
            VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')],
            PubliclyAccessible=True
        )
    print(response)
except aws_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La Instancia de Base de Datos ya Existe.")

{'DBInstance': {'DBInstanceIdentifier': 'dw-db', 'DBInstanceClass': 'db.t3.micro', 'Engine': 'mysql', 'DBInstanceStatus': 'creating', 'MasterUsername': 'root', 'DBName': 'dw', 'AllocatedStorage': 10, 'PreferredBackupWindow': '07:40-08:10', 'BackupRetentionPeriod': 1, 'DBSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0a991954ba54ddb2e', 'Status': 'active'}], 'DBParameterGroups': [{'DBParameterGroupName': 'default.mysql8.0', 'ParameterApplyStatus': 'in-sync'}], 'DBSubnetGroup': {'DBSubnetGroupName': 'default', 'DBSubnetGroupDescription': 'default', 'VpcId': 'vpc-018eccc2fe2d01a58', 'SubnetGroupStatus': 'Complete', 'Subnets': [{'SubnetIdentifier': 'subnet-0dba6b93abeb35181', 'SubnetAvailabilityZone': {'Name': 'us-east-1f'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-0bf89f2925eb7560f', 'SubnetAvailabilityZone': {'Name': 'us-east-1b'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-0d33d4f2374493feb', 'S

In [50]:
try:
     instances = aws_conn.describe_db_instances(DBInstanceIdentifier=rdsIdentifier)
     RDS_DW_HOST = instances.get('DBInstances')[0].get('Endpoint').get('Address')
     print(RDS_DW_HOST)
except Exception as ex:
     print("La instancia de base de datos no existe o aun no se ha terminado de crear.")
     print(ex)

dw-db.cxfxom467xtx.us-east-1.rds.amazonaws.com


In [51]:
import create_dw_query
import mysql.connector as mysqlC
try:
    myDw = mysqlC.connect(
    host=RDS_DW_HOST, 
    user=config.get('RDS_MYSQL', 'DB_USER'),
    password=config.get('RDS_MYSQL', 'DB_PASSWORD'),
    database=config.get('RDS_MYSQL', 'DB_NAME')
    )
    
    mycursor = myDw.cursor()
    mycursor.execute(create_dw_query.CREATE_DW, multi=True)
    
    myDw.close()
    
    print("Data Warehouse Creado Exitosamente")
except mysqlC.Error as err:
    print("Something went wrong:{}".format(err))

Data Warehouse Creado Exitosamente


#### Insertamos datos

In [52]:
mysql_driver = f"""mysql+pymysql://{config.get('RDS_MYSQL', 'DB_USER')}:{config.get('RDS_MYSQL', 'DB_PASSWORD')}@{RDS_DW_HOST}:{config.get('RDS_MYSQL', 'DB_PORT')}/{config.get('RDS_MYSQL', 'DB_NAME')}"""  

In [53]:
dimpaymentype.columns

Index(['paymentsk', 'payment_type'], dtype='object')

In [ ]:
#cambio nombre de columnas previo a insertar dentro de dimension
dimpaymentype=dimpaymentype.rename(columns={"paymentsk": "dimpaymentpk"})
#se insertan los datos
dimpaymentype.to_sql('dimpaymentype', mysql_driver, index=False, if_exists='append')

In [56]:
dimFechaHora.columns

Index(['order_purchase_timestamp', 'year', 'month', 'quarter', 'day', 'week',
       'dayofweek', 'hour', 'minute', 'is_weekend', 'dimfecha'],
      dtype='object')

In [57]:
#cambio nombre de columnas previo a insertar dentro de dimension
dimFechaHora=dimFechaHora.rename(columns={"dimfecha": "dimfechapk"})
#quito dia
dimFechaHora=dimFechaHora.loc[:, dimFechaHora.columns != 'day']
#quito duplicado por ser llave tipo string(espacios)
dimFechaHora['dimfechapk']=dimFechaHora['dimfechapk'].str.strip()
dimFechaHora=dimFechaHora.drop_duplicates(subset='dimfechapk', keep="last")


In [58]:
len(dimFechaHora['dimfechapk'])

87380

In [59]:
#se insertan los datos
dimFechaHora.to_sql('dimcalendar', mysql_driver, index=False, if_exists='append')

87380

In [60]:
#se ve nombres 
dimcustomer.columns

Index(['customersk', 'customer_id', 'name', 'email', 'phone', 'customer_city',
       'customer_state'],
      dtype='object')

In [61]:
#cambio nombre de columnas previo a insertar dentro de dimension
dimcustomer=dimcustomer.rename(columns={"customersk": "dimcustomerpk"})
#se insertan los datos
dimcustomer.to_sql('dimcustomer', mysql_driver, index=False, if_exists='append')

99441

In [62]:
#se ve nombres 
dimseller.columns

Index(['sellersk', 'seller_id', 'name', 'email', 'phone', 'seller_city',
       'seller_state'],
      dtype='object')

In [63]:
#cambio nombre de columnas previo a insertar dentro de dimension
dimseller=dimseller.rename(columns={"sellersk": "dimsellerpk"})
#se insertan los datos
dimseller.to_sql('dimseller', mysql_driver, index=False, if_exists='append')

3095

In [64]:
#se ve nombres 
dimreview.columns

Index(['reviewsk', 'review_id', 'review_score', 'review_comment_message'], dtype='object')

In [65]:
#cambio nombre de columnas previo a insertar dentro de dimension
dimreview=dimreview.rename(columns={"reviewsk": "dimreviewpk"})


In [66]:
#se insertan los datos
dimreview.to_sql('dimreview', mysql_driver, index=False, if_exists='append')

98410

In [67]:
#se ve nombres 
dimproduct.columns

Index(['productsk', 'product_id', 'product_width_cm', 'product_length_cm',
       'product_height_cm', 'product_weight_g'],
      dtype='object')

In [68]:
#cambio nombre de columnas previo a insertar dentro de dimension
dimproduct=dimproduct.rename(columns={"productsk": "dimproductpk"})
#se insertan los datos
dimproduct.to_sql('dimproduct', mysql_driver, index=False, if_exists='append')

32951

In [69]:
#se ve nombres 
dimproductcate.columns

Index(['productcatesk', 'product_category_name'], dtype='object')

In [71]:
#cambio nombre de columnas previo a insertar dentro de dimension
dimproductcate=dimproductcate.rename(columns={"productcatesk": "dimproductcatepk"})

#se insertan los datos
dimproductcate.to_sql('dimproductcate', mysql_driver, index=False, if_exists='append')

74

In [72]:
#se ve nombres 
dimorder.columns

Index(['ordersk', 'order_id', 'order_status'], dtype='object')

In [73]:
#cambio nombre de columnas previo a insertar dentro de dimension
dimorder=dimorder.rename(columns={"ordersk": "dimorderpk"})

#se insertan los datos
dimorder.to_sql('dimorder', mysql_driver, index=False, if_exists='append')

99441

In [74]:
#se ve nombres 
factTable_transacciones.columns

Index(['id_detalle', 'price', 'payment_value', 'order_item_id', 'ordersk',
       'customersk', 'paymentsk', 'reviewsk', 'productsk', 'sellersk',
       'productcatesk', 'dimfechafk'],
      dtype='object')

In [75]:
#cambio nombre de columnas previo a insertar dentro de dimension
factTable_transacciones=factTable_transacciones.rename(columns={"ordersk": "dimorderfk","customersk": "dimcustomerfk",
                                                                "paymentsk": "dimpaymentfk","reviewsk": "dimreviewfk",
                                                                "productsk": "dimproductfk","sellersk": "dimsellerfk",
                                                                "prodcutcatesk": "dimproductcatefk","id_detalle":"id_registro"})


In [76]:
factTable_transacciones['dimfechafk']=factTable_transacciones['dimfechafk'].str.strip()

In [77]:
#se insertan los datos
factTable_transacciones.to_sql('factable', mysql_driver, index=False, if_exists='append')

118307